<a href="https://colab.research.google.com/github/123ai0/samurai/blob/main/SalesForecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%pip install -Uq upgini catboost

In [9]:
from os.path import exists
import pandas as pd

df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df=df.sample(n=19_000, random_state=0)
df["store"]= df["store"].astype(str)
df["item"]=df["item"].astype(str)

df["date"]=pd.to_datetime(df["date"])

df.sort_values("date" , inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [10]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"]>= "2017-01-01"]

In [12]:
train_features = train.drop(columns=["sales"])
train_target = train["sales"]
test_features = test.drop(columns=["sales"])
test_target =test["sales"]

Enrich Features

In [14]:
from upgini import FeaturesEnricher,SearchKey
from upgini.metadata import CVType

enricher = FeaturesEnricher(
    search_keys= {
        "date": SearchKey.DATE,
    },
    cv=CVType.time_series
)
enricher.fit(train_features,
             train_target,
             eval_set=[(test_features,test_target)])

[============================================================] 100% Finished


Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION




Column name,Status,Errors
target,All valid,-
date,All valid,-



Running search request, search_id=64c8d222-fa5f-42d5-812d-616c74816b63
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com


60 relevant feature(s) found with the search keys: ['date']


f_weather_date_weather_umap_34_c3ef5b4f,0.0505,100.0000,"4.8168, 4.8289, 4.6887",Upgini,Weather & climate normals data,Daily
f_autofe_div_50507976,0.0138,100.0000,"-0.0517, 0.0002, 0.0159",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_weather_date_weather_pca_0_d7e0a1fc,0.0131,100.0000,"-29.3741, 29.8194, 28.5345",Upgini,Weather & climate normals data,Daily
f_autofe_div_029f75a6,0.0125,100.0000,"-0.3972, 0.4045, -0.7587",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_a9b56416,0.0107,100.0000,"-0.4305, -0.9139, -0.4429",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_34d11bc4,0.0107,100.0000,"13.8629, 13.2302, 7.7209",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_events_date_year_cos1_9014a856,0.0063,100.0000,"0.556, 0.6902, -0.5582",Upgini,Calendar data,Daily
f_autofe_div_50a5dcaf,0.0055,100.0000,"0.7421, -1.26, 0.8424",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_mul_478be4f3,0.0049,100.0000,"0.1107, -0.2326, -0.4506",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_89c56a5f,0.0048,100.0000,"0.9836, -0.2482, 1.2627",Upgini,"AutoFE: features from Calendar data,Markets data",Daily
f_autofe_div_3e22df83,0.0046,100.0000,"0.0228, -0.0263, 0.0091",Upgini,"AutoFE: features from Calendar data,Markets data",Daily


Upgini,"AutoFE: features from Calendar data,Markets data",0.1123,41
Upgini,Weather & climate normals data,0.0677,6
Upgini,Calendar data,0.0063,1
Upgini,AutoFE: features from Markets data,0.0035,5
Upgini,AutoFE: features from Calendar data,0.0011,3
Upgini,Markets data,0.0010,2
Upgini,World economic indicators,0.0004,2


"Calendar data,Markets data",f_autofe_div_50507976,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,/
"Calendar data,Markets data",f_autofe_div_029f75a6,f_events_date_week_sin1_847b5db1,f_financial_date_nasdaq_7d_to_7d_1y_shift_4a4dd6ce,/
"Calendar data,Markets data",f_autofe_mul_a9b56416,f_events_date_week_sin1_847b5db1,f_financial_date_natural_gas_1d_to_7d_1fd6362a,*
"Calendar data,Markets data",f_autofe_mul_34d11bc4,f_events_date_year_cos1_9014a856,f_financial_date_silver_e4e33014,*
"Calendar data,Markets data",f_autofe_div_50a5dcaf,f_events_date_week_cos3_7525fe31,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,/
"Calendar data,Markets data",f_autofe_mul_478be4f3,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,*
"Calendar data,Markets data",f_autofe_div_89c56a5f,f_events_date_year_cos1_9014a856,f_financial_date_natural_gas_7d_to_7d_1y_shift_a5c3c07f,/
"Calendar data,Markets data",f_autofe_div_3e22df83,f_events_date_week_sin1_847b5db1,f_financial_date_crude_oil_1f195998,/
"Calendar data,Markets data",f_autofe_div_380598ae,f_events_date_week_cos3_7525fe31,f_financial_date_usd_eur_7d_to_1y_200e4902,/
"Calendar data,Markets data",f_autofe_div_534d01d5,f_events_date_week_sin1_847b5db1,f_financial_date_stoxx_043cbcd4,/
"Calendar data,Markets data",f_autofe_div_c9ddcb38,f_events_date_week_cos1_f6a8c1fc,f_financial_date_silver_7d_to_7d_1y_shift_55fa8001,/



Examples of outliers with maximum value of target:
84    205
47    196
38    187
Name: target, dtype: int64
Outliers will be excluded during the metrics calculation.
Calculating accuracy uplift after enrichment...

which makes metrics between the train and eval_set incomparable.


Train,15213,50.3977,301.2609,191.5005,109.7604
Eval 1,3787,59.2424,485.3569,364.2799,121.0770


In [18]:
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

model = CatBoostRegressor(verbose=False, allow_writing_files=False, random_state=0)

enricher.calculate_metrics(
    train_features,train_target,
    eval_set=[(test_features, test_target)],
    estimator=model,
    scoring="mean_absolute_percentage_error"
)

Calculating accuracy uplift after enrichment...
-
which makes metrics between the train and eval_set incomparable.


,Dataset type,Rows,Mean target,Baseline mean_absolute_percentage_error,Enriched mean_absolute_percentage_error,Uplift
0,Train,15213,50.3977,0.252850,0.159122,0.093728
1,Eval 1,3787,59.2424,0.264187,0.204403,0.059784


In [24]:
enriched_train_features = enricher.transform(train_features, keep_input=True)
enriched_test_features = enricher.transform(test_features, keep_input=True)
enriched_train_features.head()

[============================================================] 100% Finished

You are trying to launch enrichment for 15213 rows, which will exceed the rest limit 10000.


Button(button_style='danger', description='Request a quote', style=ButtonStyle())

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 10000 rows. You have already enriched: 0 rows.

Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IPv4 to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-



Running search request, search_id=8efcc253-49a4-4ffe-8081-1ff0116af124
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


AttributeError: 'NoneType' object has no attribute 'head'